In [1]:
# load face embeddings
import csv
import numpy as np

names = []
faces = []

with open('./output/train_vgg_face.csv', 'r') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        names.append(row[0])
        faces.append(np.array(row[1:]).astype(float))
        
print(len(faces), 'training faces')

105 training faces


In [7]:
# labels for classes
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
labels = le.fit_transform(names)
for i in range(le.classes_.shape[0]):
    print(i, '>>', le.classes_[i])

0 >> chiao
1 >> kaze
2 >> kiki
3 >> real
4 >> ricebug
5 >> tsungchan
6 >> tsungyu


In [13]:
# train svm 

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn.svm import SVC

# grid search for parameters with 10-fold cross validation
for C in [1, 10, 100, 1000, 10000]:
    for gamma in [0.00001, 0.0001, 0.001, 0.01, 0.1]:
        svc = SVC(C=C, gamma=gamma, kernel='rbf', probability=False)
        cv = ShuffleSplit(n_splits=10, test_size=0.2)
        scores = cross_val_score(svc, faces, labels, cv=cv)
        print("C = {0:5d}, gamma = {1:1.6f}, score = {2:1.3f}".format(C, gamma, np.mean(scores)))

C =     1, gamma = 0.000010, score = 0.105
C =     1, gamma = 0.000100, score = 0.848
C =     1, gamma = 0.001000, score = 0.881
C =     1, gamma = 0.010000, score = 0.048
C =     1, gamma = 0.100000, score = 0.052
C =    10, gamma = 0.000010, score = 0.886
C =    10, gamma = 0.000100, score = 0.919
C =    10, gamma = 0.001000, score = 0.838
C =    10, gamma = 0.010000, score = 0.071
C =    10, gamma = 0.100000, score = 0.062
C =   100, gamma = 0.000010, score = 0.929
C =   100, gamma = 0.000100, score = 0.919
C =   100, gamma = 0.001000, score = 0.876
C =   100, gamma = 0.010000, score = 0.071
C =   100, gamma = 0.100000, score = 0.048
C =  1000, gamma = 0.000010, score = 0.929
C =  1000, gamma = 0.000100, score = 0.933
C =  1000, gamma = 0.001000, score = 0.843
C =  1000, gamma = 0.010000, score = 0.062
C =  1000, gamma = 0.100000, score = 0.062
C = 10000, gamma = 0.000010, score = 0.914
C = 10000, gamma = 0.000100, score = 0.914
C = 10000, gamma = 0.001000, score = 0.910
C = 10000, 

In [16]:
# set the parameters we want and train 
import pickle

svc = SVC(C=1000, gamma=0.0001, kernel='rbf', probability=False)
svc.fit(faces, labels)

with open('./output/svc_vgg_face.model', 'wb') as f:
    pickle.dump(svc, f)